# STEP 0: Install dependencies

In [ ]:
!pip install -qu google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15\
langgrapgh langchain langchahan-google-genai-genai openai

In [ ]:

!pip install langgraph langchain langchain-core
!pip install langchain-google-genai



# STEP 1: Imports and secure API key input

In [ ]:
import os
import getpass #enable us to get pass to apikey noone will see apikey
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain_core.messages import HumanMessage


# Secure Gemini API Key input

In [ ]:
os.environ['Google_API_KEY']=getpass.getpass('Enter Google API Key: ')

# STEP 2: Initialize Gemini 1.5 Flash

In [ ]:
llm=ChatGoogleGenerativeAI(model= "model/gemini-1.5-flash-latest", temperature=0.2)#temperature will controll the randomness of llm. HIGHER=CREATIVE, LOWER=REPETITIVE

# STEP 3: Node to ask user for symptom

In [ ]:
def get_symptom(state:dict)-> dict:
  symptom= input("Welcome to Dr. Singh Hospital, Please let us know your symptoms")
  state['symptom']=symptom
  return state

# STEP 4: Node to classify the symptom

In [ ]:
def classify_symptom(state:dict)-> dict:
  prompt=(
      "You are a helpful medical Assistant, Classify the symptoms below into below one of the categores\n"
      "-General\n -Emergency n\ -mental health \n"
      f"symptom :{state['symptom']} \n"
      "Respond only with one word : General, Emergency or Mental Health"
      "#Example : input: I have fever, output: General"

  )
  response=llm.invoke([HumanMessage(content=prompt)])
  category= response.content.strip() #taking message from the llm into STRING format
  print(f"LLM classify the sympotom as :{category}") #debug
  state['category']=category
  return state

# STEP 5: Router logic to route to the correct node


In [ ]:

def symptom_router(state:dict)-> dict:
  category = state['category'].lower()
  if category == 'Emergency':
    return 'Emergency'
  elif category == 'Mental Health':
    return 'Mental Health'
  else:
    return 'General'

# STEP 6: Category-specific response nodes


In [ ]:
def general_node(state:dict)-> dict:
  state["answer"]=f"'{state['symptom']}': seems general : directing you to the general ward for consulting a doctor"
  return state

def emergency_node(state:dict)-> dict:
  state["answer"]=f"'{state['symptom']}': seems urgent medical emergency : directing you to the emergency ward for consulting a doctor"
  return state

def mental_health_node(state:dict)-> dict:
  state["answer"]=f"'{state['symptom']}': seems mental heath issue : directing you to the mental health ward for consulting a doctor"
  return state

# STEP 7: Build LangGraph


In [ ]:
builder= StateGraph(dict)

#define the nodes
builder.set_entry_point("get_symptom")
builder.add_node("get_symptom",get_symptom)
builder.add_node("classify_symptom",classify_symptom)
builder.add_node("general",general_node)
builder.add_node("Emergency",emergency_node)
builder.add_node("Mental Health",mental_health_node)

builder.add_edge("get_symptom","classify_symptom")
builder.add_conditional_edges("classify_symptom",symptom_router,{
    "General": "general",
    "Emergency": "Emergency",
    "Mental Health": "Mental Health"
})

builder.add_edge("general",END)
builder.add_edge("Emergency",END)
builder.add_edge("Mental Health",END)



# STEP 8: Compile and invoke the graph


In [ ]:
# Define the initial state (inputs)
initial_state = {"symptom": "I have a headache and a sore throat."}


# Invoke the graph
output = graph.invoke(initial_state)


# Print the final state
print("final output: \n")
print(output["answer"])